In [1]:
import warnings 
warnings.simplefilter('ignore')

import scanpy as sc
import anndata
import scparadise
import pandas as pd
import muon as mu
import numpy as np
import pandas as pd
import os
import scvi
import torch

In [2]:
os.chdir('/mnt/c/Users/vadim/scRNA/scParadise/scripts_article')

In [5]:
# Load data
adata = sc.read_h5ad('Pancreas/adata.h5ad') 

AnnData object with n_obs × n_vars = 245878 × 27484
    obs: 'Annotated_Clusters', 'LibraryID', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'Seurat_Clusters', 'BMI', 'HbA1c', 'Islet', 'UNOS', 'Center', 'Medication', 'Viability', 'Purity', 'Cause_of_death', 'Multiplex', 'BMI_Category', 'Clusters_DiseaseState', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'suspension_type', 'is_primary_data', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'batch_condition', 

In [24]:
# Create training and testing datasets
adata_train = adata[adata.obs['donor_id'].isin(['SAMN16365027', 'SAMN17528599'])].copy()
adata_test_1 = adata[adata.obs['donor_id'].isin(['HP-20152-01'])].copy()
adata_test_2 = adata[adata.obs['donor_id'].isin(['HP17225-01T2D'])].copy()
adata_test_3 = adata[adata.obs['donor_id'].isin(['HP19047-01'])].copy()
adata_test_4 = adata[adata.obs['donor_id'].isin(['SAMN11867362'])].copy()
adata_test_5 = adata[adata.obs['donor_id'].isin(['SAMN12022246'])].copy()
adata_test_6 = adata[adata.obs['donor_id'].isin(['SAMN13108021', 'SAMN11642375'])].copy()
adata_test_7 = adata[adata.obs['donor_id'].isin(['SAMN11476721', 'SAMN10873960'])].copy()
adata_test_8 = adata[adata.obs['donor_id'].isin(['SAMN10439569', 'SAMN10737781'])].copy()
adata_test_9 = adata[adata.obs['donor_id'].isin(['SAMN13319813', 'SAMN11522709'])].copy()
adata_test_10 = adata[adata.obs['donor_id'].isin(['HP18304', 'SAMN11157311'])].copy()

# Normalize data, find highly variable features
for i in [adata_train, adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8, adata_test_9, adata_test_10]:
    i.layers['counts'] = i.X.copy()
    sc.pp.normalize_total(i, target_sum=None)
    sc.pp.log1p(i)
    i.raw = i

In [25]:
# Subset anndata object based on a selected marker genes
genes = pd.read_csv('Pancreas/genes_for_AI.csv')
adata_train = adata_train[:, genes.genes].copy()

In [30]:
# Train scvi model for adata_train dataset
scvi.model.SCVI.setup_anndata(adata_train, batch_key="donor_id", layer="counts")
scvi_ref = scvi.model.SCVI(adata_train)
scvi_ref.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████████| 400/400 [05:00<00:00,  1.27it/s, v_num=1, train_loss_step=225, train_loss_epoch=225]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████████| 400/400 [05:00<00:00,  1.33it/s, v_num=1, train_loss_step=225, train_loss_epoch=225]


In [12]:
scvi_ref_path = 'Pancreas/scanvi_test/models'
scvi_ref.save(scvi_ref_path, overwrite=True)

In [27]:
# Create lists with test samples
lst_test_folders = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
lst_test_adatas = [adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8, adata_test_9, adata_test_10]
lst_annotations = ['celltype_l1']

In [28]:
for folder in lst_test_folders:
    os.makedirs(os.path.join('Pancreas/scanvi_test/reports', folder))
for i in lst_annotations:
    os.makedirs(os.path.join('Pancreas/scanvi_test/models', i))

In [31]:
for i in lst_annotations:
    # Train scanvi model for reference dataset (adata_train)
    scanvi_ref = scvi.model.SCANVI.from_scvi_model(
        scvi_ref,
        unlabeled_category="Unknown",
        labels_key=i,
    )
    # Train scanvi models for selected annotation level
    scanvi_ref.train()
    scanvi_ref_path = os.path.join('Pancreas/scanvi_test/models', i)
    scanvi_ref.save(scanvi_ref_path, overwrite=True)
    
    for adata_test, folder in zip(lst_test_adatas, lst_test_folders):
        # Train scanvi model for test dataset (adata_test)
        scvi.model.SCANVI.prepare_query_anndata(adata_test, scanvi_ref)
        scanvi_test = scvi.model.SCANVI.load_query_data(adata_test, scanvi_ref)
        scanvi_test.train(
            max_epochs=10
        )
        # Create name for predicted annotation level
        SCANVI_annotation = 'scanvi_' + i
        # Predict annotation using pretrained scanvi_test model
        adata_test.obs[SCANVI_annotation] = scanvi_test.predict()
        # Create and save classification report of selected annotation level
        file_save = 'report_test_scanvi_default_' + i + '.csv'
        scparadise.scnoah.report_classif_full(adata_test, 
                                              celltype = i, 
                                              pred_celltype = SCANVI_annotation, 
                                              report_name = file_save,
                                              save_path = os.path.join('Pancreas/scanvi_test/reports', folder),
                                              save_report=True)

INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:18<00:00,  1.85s/it, v_num=1, train_loss_step=213, train_loss_epoch=209]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:18<00:00,  1.84s/it, v_num=1, train_loss_step=213, train_loss_epoch=209]
INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


...

Epoch 10/10: 100%|██████████████████| 10/10 [00:29<00:00,  2.98s/it, v_num=1, train_loss_step=210, train_loss_epoch=214]
Successfully saved report

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:14<00:00,  1.41s/it, v_num=1, train_loss_step=203, train_loss_epoch=206]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:14<00:00,  1.41s/it, v_num=1, train_loss_step=203, train_loss_epoch=206]
Successfully saved report

